## PART 1: PHPMyAdmin#

**Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.**

In [ ]:
SELECT name  
FROM Facilities  
WHERE membercost = 0.0;

**Q2: How many facilities do not charge a fee to members?**

4 Facilities:  
- Badminton Court   
- Table Tennis  
- Snooker Table  
- Pool Table

**Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.**

In [ ]:
SELECT facid, name, membercost, monthlymaintenance  
FROM Facilities  
WHERE membercost < (monthlymaintenance * .2);  

**Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.**

In [ ]:
SELECT *  
FROM Facilities  
WHERE facid IN (1, 5);  

**Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.**

In [ ]:
SELECT name, monthlymaintenance,   
CASE   
    WHEN monthlymaintenance < 100 THEN 'cheap'  
    ELSE 'expensive' END AS cheap_or_expensive  
FROM Facilities;  

**Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.**

In [ ]:
SELECT surname, firstname, joindate
FROM Members
ORDER BY joindate DESC;

**Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.**

In [ ]:
SELECT b.bookid, f.name, CONCAT(m.surname, ' ', m.firstname) AS member,
CASE
	WHEN b.memid = 0 THEN b.slots*f.guestcost
	ELSE b.slots*f.membercost END AS cost
FROM Bookings AS b
LEFT JOIN Facilities AS f
USING (facid)
LEFT JOIN Members AS m
ON b.memid = m.memid
WHERE b.starttime >= '2012-09-14' 
AND b.starttime < '2012-09-15'
AND b.memid != 0 
AND f.membercost * slots > 30 
OR b.starttime >= '2012-09-14' 
AND b.starttime < '2012-09-15'
AND b.memid = 0
AND f.guestcost * slots > 30
ORDER BY cost;

**Q9: This time, produce the same result as in Q8, but using a subquery.**

In [ ]:
SELECT b.bookid, f.name, CONCAT(m.surname, ' ', m.firstname) AS member,
CASE
	WHEN b.memid = 0 THEN b.slots*f.guestcost
	ELSE b.slots*f.membercost END AS cost
FROM 
	(SELECT *
     FROM Bookings
     WHERE starttime >= '2012-09-14' 
	 AND starttime < '2012-09-15') AS b
LEFT JOIN Facilities AS f
USING (facid)
LEFT JOIN Members AS m
ON b.memid = m.memid
WHERE b.memid != 0 
AND f.membercost * slots > 30 
OR b.memid = 0
AND f.guestcost * slots > 30
ORDER BY cost;

## PART 2: SQLite

In [8]:
# Import packages
import sqlite3
import pandas as pd

In [16]:
# Create engine and test connection
con = sqlite3.connect('sqlite_db_pythonsqlite.db')

def sql_fetch(con):
    cur = con.cursor()
    cur.execute("SELECT * FROM Facilities")
    rows = cur.fetchall()
    for row in rows:
        print(row)
        
sql_fetch(con)

(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


**Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!**

In [57]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
query = 'SELECT name, revenue \
FROM (SELECT b.facid, f.name, \
SUM(CASE WHEN b.memid = 0 THEN b.slots*f.guestcost \
ELSE b.slots*f.membercost END) AS revenue \
FROM bookings AS b \
LEFT JOIN facilities AS f \
USING (facid) \
GROUP BY b.facid) \
WHERE revenue < 1000 \
ORDER BY revenue'

with con:
    cur = con.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    q10 = pd.read_sql_query(query, con)
    
con.close()

In [58]:
q10

,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


**Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order**

In [77]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
query = 'SELECT m1.surname, m1.firstname, m2.surname || " " || m2.firstname AS recommendedby \
FROM members AS m1 \
LEFT JOIN members AS m2 \
ON m1.recommendedby = m2.memid \
ORDER BY m1.surname, m1.firstname'

with con:
    cur = con.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    q11 = pd.read_sql_query(query, con)
    
con.close()

In [78]:
q11.head(10)

,surname,firstname,recommendedby
0,Bader,Florence,Stibbons Ponder
1,Baker,Anne,Stibbons Ponder
2,Baker,Timothy,Farrell Jemima
3,Boothe,Tim,Rownam Tim
4,Butters,Gerald,Smith Darren
5,Coplin,Joan,Baker Timothy
6,Crumpet,Erica,Smith Tracy
7,Dare,Nancy,Joplette Janice
8,Farrell,David,None
9,Farrell,Jemima,None


**Q12: Find the facilities with their usage by member, but not guests**

In [82]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
query = 'SELECT f.name, SUM(b.slots*f.membercost) AS memberusage \
FROM bookings AS b \
LEFT JOIN facilities AS f \
USING (facid) \
GROUP BY b.facid'

with con:
    cur = con.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    q12 = pd.read_sql_query(query, con)
    
con.close()

In [84]:
q12

,name,memberusage
0,Tennis Court 1,6600.0
1,Tennis Court 2,6390.0
2,Badminton Court,0.0
3,Table Tennis,0.0
4,Massage Room 1,13899.6
5,Massage Room 2,2257.2
6,Squash Court,3864.0
7,Snooker Table,0.0
8,Pool Table,0.0


**Q13: Find the facilities usage by month, but not guests**

In [86]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
query = "SELECT f.name, strftime('%m', b.starttime) AS month, SUM(b.slots*f.membercost) AS memberusage \
FROM bookings AS b \
LEFT JOIN facilities AS f \
USING (facid) \
GROUP BY b.facid, month"

with con:
    cur = con.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    q13 = pd.read_sql_query(query, con)
    
con.close()

In [87]:
q13

,name,month,memberusage
0,Tennis Court 1,07,1350.0
1,Tennis Court 1,08,2295.0
2,Tennis Court 1,09,2955.0
3,Tennis Court 2,07,1035.0
4,Tennis Court 2,08,2415.0
5,Tennis Court 2,09,2940.0
6,Badminton Court,07,0.0
7,Badminton Court,08,0.0
8,Badminton Court,09,0.0
9,Table Tennis,07,0.0
